# Set up

In [1]:
!git clone https://github.com/NLP-Reichman/2025_assignment_1.git
!mv 2025_assignment_1/data data
!rm 2025_assignment_1/ -r

Cloning into '2025_assignment_1'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 71 (delta 24), reused 44 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 7.05 MiB | 10.91 MiB/s, done.
Resolving deltas: 100% (24/24), done.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [2]:
import os
import json
from google.colab import files
import pandas as pd
import numpy as np
from itertools import product

is_using_colab = os.getenv("COLAB_RELEASE_TAG")
if is_using_colab:
    from google.colab import files

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data. The vocabulary should include the `<start>` and  `<end>` tokens.

Note - do NOT lowecase the sentences.

In [3]:
from collections import defaultdict, Counter

def load_data():
    directory = "data"
    tweets = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                data = json.load(file)
            language = os.path.splitext(filename)[0]
            raw_tweets = data.get("tweet_text", {})
            # tweet_texts = clean_data(raw_tweets)
            # for tweet_id, text in tweet_texts.items():
            for tweet_id, text in raw_tweets.items():
                tokens = tokenize_text(text)
                tweets.append([tokens, language])
    return np.array(tweets, dtype=object)

def tokenize_text(text):
    return list(text)

tweets = load_data()

In [4]:
def preprocess() -> list[str]:
    '''
    Return a list of characters, representing the shared vocabulary of all languages
    '''
    vocab = []
    # build the global character‑level vocabulary once
    vocab_counter = Counter()
    for tokens, _lang in tweets:
        vocab_counter.update(tokens)

    vocab_counter.update(['<start>', '<end>'])
    vocab = list(vocab_counter.keys())
    return vocab

# We didn't want to change the function signature so we split it into separate helper function
def preprocess_one_lang(tweets, n) -> list[str]:
    '''
    Given a word-separated ndarray of size (N,), returns vocabulary for this ndarray
    Note: Here we assume all tweets are in the same language.

    n: the sentence length (for the n-gram)
    '''
    vocab_counter = Counter()
    processed_tweets = []
    for tweet_text in tweets:
        tweet_text = ['<start>'] * (n - 1) + tweet_text + ['<end>']
        processed_tweets.append(tweet_text)
        vocab_counter.update(tweet_text)
    vocab_counter.most_common()
    vocab = list(vocab_counter.keys())
    return vocab, processed_tweets

In [5]:
vocab = preprocess()
print(f"vocab length: {len(vocab)}")
print(f"Some characters in the vocab: {vocab[:10]}")

vocab length: 1804
Some characters in the vocab: ['R', 'T', ' ', '@', 'A', 'z', 'u', '_', 'M', 'o']


## Part 2
Implement the function *build_lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. To ensure consistent probabilities calculation, please add n-1 `<start>` tokens to the beginning of a tweet and one `<end>` token at the end. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add-one smoothing information to the dictionary and implement it.

Please add the `<unk>` token with $p(<unk>)=1/|V|$ to the LM if buiulding a smoothed LM.

In [6]:
def build_lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :param smoothed: boolean indicating whether to apply smoothing
    :return: a dictionary where the keys are n_grams and the values are dictionaries
    '''
    LM = {}
    # ---------- memoize ----------
    if not hasattr(build_lm, "_cache"):
        build_lm._cache = {}
    ckey = (lang, n, smoothed)
    if ckey in build_lm._cache:
        return build_lm._cache[ckey]
    # ---------- gather & pad tweets ----------
    tweet_texts = tweets[tweets[:, 1] == lang, 0]
    if len(tweet_texts) == 0:
        raise ValueError("No such language!")
    lang_vocab, processed_tweets = preprocess_one_lang(tweet_texts, n)
    V = len(lang_vocab)
    # ---------- count n‑grams ----------
    lm_counts = defaultdict(Counter)
    for chars in processed_tweets:                 # chars is list[str]
        for i in range(len(chars) - (n - 1)):
            key = "".join(chars[i : i + n - 1])    # (n‑1)‑gram context
            next_c = chars[i + n - 1]
            lm_counts[key][next_c] += 1

    # ---------- store probabilities (observed only) ----------
    for ctx, cnts in lm_counts.items():
        total = sum(cnts.values())
        denom = total + (V if smoothed else 0)
        LM[ctx] = {c: (k + (1 if smoothed else 0)) / denom for c, k in cnts.items()}

    # constant unknown prob for *any* unseen event in this context
    if smoothed:
        LM["<unk>"] = 1 / V

    build_lm._cache[ckey] = LM
    return LM

In [7]:
LM = build_lm("en", 3, False)
print(f"English Language Model with 3-gram is of length: {len(LM)}")

English Language Model with 3-gram is of length: 8238


In [8]:
LM2 = build_lm("en", 3, True)
print(f"English Language Model with 3-gram smoothed is of length: {len(LM2)}")

English Language Model with 3-gram smoothed is of length: 8239


## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

The `<unk>` should be used for unknown contexts when calculating the perplexities.

In [9]:
def perplexity(model: dict, text: list, n) -> float:
  '''
  Calculates the perplexity of the given string using the given language model.
  :param model: The language model
  :param text: The tokenized text to calculate the perplexity for
  :param n: The n-gram of the model
  :return: The perplexity
  '''
  pp = 0
  unk_prob_global = model.get("<unk>", 1e-8)

  log_sum = 0.0
  steps   = 0

  for i in range(len(text) - (n - 1)):
      ctx  = "".join(text[i : i + n - 1])
      nxt  = text[i + n - 1]
      prob = model.get(ctx, {}).get(nxt, unk_prob_global)
      log_sum += np.log(prob)
      steps  += 1

  pp = np.exp(-log_sum / max(steps, 1))
  return pp

In [10]:
def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :param n: The n-gram of the model
  :return: the perplexity value
  '''
  pp = 0
  # cache joined & padded text per (lang,n)
  if not hasattr(eval, "_cache"):
      eval._cache = {}
  ckey = (target_lang, n)
  if ckey not in eval._cache:
      tgt_texts = tweets[tweets[:, 1] == target_lang, 0]
      _, proc = preprocess_one_lang(tgt_texts, n)
      eval._cache[ckey] = [ch for sent in proc for ch in sent]

  pp = perplexity(model, eval._cache[ckey], n)
  return pp

In [11]:
LM = build_lm("en", 3, True)

In [12]:
print("Perplexity of the English 3-gram model on datasets:")
print(f"On English: {eval(LM, 'en', 3): .2f}")
print(f"On French: {eval(LM, 'fr', 3): .2f}")
print(f"On Dutch: {eval(LM, 'nl', 3): .2f}")
print(f"On Tagalog: {eval(LM, 'tl', 3): .2f}")


Perplexity of the English 3-gram model on datasets:
On English:  19.59
On French:  39.10
On Dutch:  41.58
On Tagalog:  47.18


In [13]:
lm1 = build_lm("en", 1, True)
lm2 = build_lm("en", 2, True)
lm3 = build_lm("en", 3, True)
lm4 = build_lm("en", 4, True)

print("Perplexity on differnet n-gram models on English")
print(f"On 1-gram: {eval(lm1, 'en', 1): .2f}")
print(f"On 2-gram: {eval(lm2, 'en', 2): .2f}")
print(f"On 3-gram: {eval(lm3, 'en', 3): .2f}")
print(f"On 4-gram: {eval(lm4, 'en', 4): .2f}")

Perplexity on differnet n-gram models on English
On 1-gram:  37.82
On 2-gram:  20.48
On 3-gram:  19.59
On 4-gram:  34.37


## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [14]:
languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']

In [15]:
def match() -> pd.DataFrame:
    ''''
    Return a DataFrame containing one line per every language pair and n_gram.
    Each line will contain the perplexity calculated when applying the language model
    of the source language on the text of the target language.
    :return: a DataFrame containing the perplexity values
    '''
    df  = pd.DataFrame()
    records = []

    for n in range(1, 5):
        # build each source model once per n
        src_models = {s: build_lm(s, n, smoothed=True) for s in languages}

        for src, tgt in product(languages, repeat=2):
            pp = eval(src_models[src], tgt, n)
            records.append((src, tgt, n, pp))

    df = pd.DataFrame(records,
                      columns=["source", "target", "n", "perplexity"])
    return df


## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token. For this exercise, you may assume a valid input in the respective language.

Note - The generation of the next token should be from the LM's distribution with NO smoothing.

In [16]:
models = {(language, n): build_lm(language, n, False)
          for language in languages
          for n in [1, 2, 3, 4]}

vocabs = {
    lang: preprocess_one_lang(tweets[tweets[:, 1] == lang, 0], 1)[0]
    for lang in languages
}

In [17]:
import random

In [18]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  text = ""
  ########################################
  text = prompt
  random.seed(r)

  model = models.get((lang, n))
  padded_prompt = ['<start>'] * (n - 1) + tokenize_text(prompt)

  while number_of_tokens > 0:
    number_of_tokens -= 1
    ngram = ''.join(padded_prompt[-(n-1):]) if (n-1) > 0 else ''
    prob_dist = model.get(ngram)
    keys = list(prob_dist.keys()) if prob_dist is not None else None
    probabilities = list(prob_dist.values()) if prob_dist is not None else None

    rand_num = random.random()
    cumulative_prob = 0.0
    if prob_dist is not None:
      for key, prob in zip(keys, probabilities):
        cumulative_prob += prob
        if rand_num < cumulative_prob:
            text += key
            break
    else: # if unrecognized ngram, we fall to lesser ngram, otherwise if n==0 we halt generation
      n -= 1
      if n == 0:
        return text
      model = models.get((lang, n)) # update used model to smaller ngram version
      number_of_tokens += 1 #didnt generate token yet, so return it to counter

    padded_prompt[-1] = key
  return text

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [19]:
print(generate('en', 1, "I am", 10, 5))
print(generate('en', 2, "I am", 10, 5))
print(generate('en', 3, "I am", 10, 5))
print(generate('en', 4, "I am ", 10, 5))
print(generate('es', 2, "Soy", 10, 5))
print(generate('es', 3, "Soy", 10, 5))
print(generate('fr', 2, "Je suis", 10, 5))
print(generate('fr', 3, "Je suis", 10, 5))

I amtpgLpC eLh
I amoulpeginSh
I amittchCalul
I am cannTacca 
Soycalíodenye
Soy (?" y qAP
Je suis:/opapropa
Je suis nvésmpe..


# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [20]:
########################################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_build_lm():
    return {
        'english_2_gram_length': len(build_lm('en', 2, True)),
        'english_3_gram_length': len(build_lm('en', 3, True)),
        'french_3_gram_length': len(build_lm('fr', 3, True)),
        'spanish_3_gram_length': len(build_lm('es', 3, True)),
    }

def test_eval():
    lm = build_lm('en', 3, True)
    return {
        'en_on_en': round(eval(lm, 'en', 3), 2),
        'en_on_fr': round(eval(lm, 'fr', 3), 2),
        'en_on_tl': round(eval(lm, 'tl', 3), 2),
        'en_on_nl': round(eval(lm, 'nl', 3), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

# TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]
TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
if is_using_colab:
    files.download('results.json')
########################################

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

total 12
drwxr-xr-x 2 root root 4096 May  4 19:53 data
-rw-r--r-- 1 root root  901 May  4 20:00 results.json
drwxr-xr-x 1 root root 4096 Apr 30 13:37 sample_data
